In [ ]:
#| default_exp nb

In [ ]:
#| export
from __future__ import annotations

# Notebook objects

> Notebook cells and outputs and helpers.



<!-- # Prologue -->

In [ ]:
#| export
import collections
import inspect
import operator as op
from copy import deepcopy
from functools import cache
from functools import partial
from typing import Callable
from typing import Iterable
from typing import Literal
from typing import Mapping
from typing import overload
from typing import Protocol
from typing import runtime_checkable
from typing import Sequence
from typing import SupportsIndex
from typing import TypeAlias

import fastcore.all as FC
from fastcore.all import L
from fastcore.xml import to_xml
from fasttransform import Transform
from nbdev.process import extract_directives
from nbdev.showdoc import add_docs
from pote.basic import empty
from pote.basic import gets
from pote.callback import Callback
from pote.callback import CollBack
from pote.callback import FuncCB
from pote.common import AD
from pote.common import update_
from pote.common import val_atpath
from pote.common import vals_at
from rich.pretty import pretty_repr


In [ ]:
#| export
import bridget.fasthtml_patching
from bridget.helpers import cached_property
from bridget.helpers import compose_first
from bridget.helpers import DetailsJSON
from bridget.helpers import emptyd


In [ ]:
#| hide
import json
from datetime import datetime
from operator import attrgetter
from pathlib import Path
from types import MethodType
from typing import Any
from typing import cast

import nbformat
import toolz as TZ
import traitlets as T
from fastcore.all import nested_idx
from fastcore.test import *
from IPython.display import display
from nbdev.showdoc import *
from pote.basic import pops_values_
from pote.callback import process_
from pote.common import shorten
from pote.common import shortens
from pote.common import val_at
from pote.display import RenderJSON
from pote.test import test_raises
from rich.console import Console


In [ ]:
#| hide
import bridget
from bridget.helpers import bridge_cfg
from bridget.helpers import in_vscode_notebook
from bridget.helpers import ts


In [ ]:
#| hide
from fasthtml.components import Div, P, Pre, Text, Span, show, B, Pre, A, Brƒ

In [ ]:
#| hide
cprint = (console := Console(width=120)).print
in_vscode = in_vscode_notebook()

In [ ]:
bridge_cfg.auto_show = True

# Notebook data

Example nbformat state:

```json
{
    "type": "state",  // "state" | ...
    "cells": [
        // Markdown cell
        {
            "cell_type": 1,  // 1: markdown, 2: code
            "source": "string",  // Cell content
            "metadata"?: {  // Optional
                "tags": ["string"],
                "jupyter": { /* jupyter specific */ },
                "brd": {
                    "id": "string", 
                    "renderer": bool  // optional
                }
            },
            "id": "string"
        },
        // Code cell with outputs
        {
            "cell_type": 2,
            "source": "string",
            "outputs"?: [  // Optional, only for code cells
                {
                    "output_type": 1 | 2 | 3 | 4 // "stream" | "display_data" | "execute_result" | "error",
                    "metadata"?: {...},  // Optional
                    // Type-specific fields
                    "name"?: "stdout" | "stderr",  // For stream
                    "text"?: "string",             // For stream
                    "data"?: {  // Optional
                        "mime/type": "string"  // e.g., "text/plain": "content"
                    },
                    "ename"?: "string",            // For error
                    "evalue"?: "string",           // For error
                    "traceback"?: ["string"],      // For error
                    "execution_count"?: number     // For execute_result
                }
            ]
        }
    ]
}
```

In [ ]:
#| output: false

test_fn = ( '../packages/nbinspect-vscode/test/outputs.json'
            if in_vscode else 
            '../packages/nbinspect-lab/nbs/outputs.json')
test_json = Path(test_fn).read_text('utf-8')
state = json.loads(test_json)

# cprint(state, width=120, overflow='ellipsis')
RenderJSON(state, init_level=2, max_height=400).display()

In [ ]:
#| output: false

def f(c):
    cc = deepcopy(c)
    if in_vscode:
        brd = cc['metadata']['brd']
        # vscode cell_id is not well formed nbformat's cell id
        brd['cell_id'], cc['id'] = cc['id'], brd['id']
        del brd['id']
    del cc['idx']
    return cc

cells = map(f, state['cells'])
md = state['nbData']['metadata']
nb_md = {
    'kernel_info': md['metadata']['kernelspec'],
}
if 'language_info' in md['metadata']: nb_md['language_info'] = md['metadata']['language_info']
nb = {
	'cells': list(cells)[:],
	'metadata': nb_md,
	'nbformat': md['nbformat'],
	'nbformat_minor': 5 # md['nbformat_minor']
}
RenderJSON(nb, init_level=2, max_height=400).display()

In [ ]:
nb = nbformat.reads(json.dumps(nb), as_version=nbformat.NO_CONVERT, capture_validation_error=(derr := {}))
test_eq(derr, {})

# NBCell

Note: though using `AD` for convenience, `NBCell` should be considered immutable.


In [ ]:
#| export

CellTypesT: TypeAlias = Literal['code', 'markdown']

copycell = op.methodcaller('copy')

def _relevant_kw(o, kw:Mapping):
    ks = tuple(FC.flatten(map(inspect.get_annotations, type(o).mro())))
    return FC.filter_keys(kw, FC.in_(ks))  # type: ignore


class NBCell(AD, metaclass=FC.NewChkMeta):
    idx: str
    cell_type: str
    source: str
    metadata: dict
    id: str
    def __new__(cls, cell: Mapping):
        if cls is NBCell: cls = _CTYP[cell.get('cell_type', 'raw')]
        return AD.__new__(cls, cell)
    def __init__(self, cell: Mapping):
        super().__init__(_relevant_kw(self, cell))
        self.source = ''.join(getattr(self, 'source', ()))
    def __ft__(self): return DetailsJSON(self, openall=True).__ft__()
    def _repr_markdown_(self): 
        return f'> {self.get('cell_type', 'raw')}\n\n```json\n{pretty_repr(self, indent_size=2, max_width=120)}\n```'
    def _repr_html_(self): return to_xml(DetailsJSON(self, summary=f"NBCell@{self.idx}", openall=True, skip=('application/json', 'metadata')))
    def copy(self) -> NBCell: return NBCell(dict(self))
    
    @cached_property
    def directives_(self) -> dict[str, list[str]]: return extract_directives(self, False)
    def has_directive(self, directive: str, *args): return has_directive(self, directive, *args)
    @cached_property
    def hidden(self): return has_directive(self, 'hide') or has_directive(self, 'include', 'false')


class NBCellRaw(NBCell): attachments: dict
class NBCellMarkdown(NBCellRaw):...
class NBCellCode(NBCell): 
    outputs: list[NBOutput]
    execution_count: int | None
    def __init__(self, cell: Mapping):
        super().__init__(cell)
        self.outputs = list(NBOutput(o) for o in cell.get('outputs', []))


class NBOutput(AD, metaclass=FC.NewChkMeta): 
    output_type: str
    def __new__(cls, out: Mapping):
        if cls is NBOutput: cls = _OUTTYP[out.get('output_type', 'stream')]
        return AD.__new__(cls, out)
    def __init__(self, out: Mapping):
        ks = tuple(FC.flatten(map(inspect.get_annotations, type(self).mro())))
        out = FC.filter_keys(out, FC.in_(ks))  # type: ignore
        super().__init__(out)

class NBOutputStream(NBOutput):
    name: Literal['stdout', 'stderr']
    text: str

class NBOutputDisplayData(NBOutput):
    data: dict
    metadata: dict

class NBOutputExecuteResult(NBOutput):
    execution_count: int
    data: dict
    metadata: dict

class NBOutputError(NBOutput):
    ename: str
    evalue: str
    traceback: list[str]


_CTYP = {'raw':NBCellRaw, 'markdown':NBCellMarkdown, 'code':NBCellCode}
# _CTYPInv = {v:k for k,v in _CTYP.items()}
_OUTTYP = {'stream':NBOutputStream, 'display_data':NBOutputDisplayData, 'execute_result':NBOutputExecuteResult, 'error':NBOutputError}
# _OUTTYPInv = {v:k for k,v in _OUTTYP.items()}

def has_directive(c: NBCell, directive: str, *args):
    return directive in c.directives_ and c.directives_[directive] == list(args)

In [ ]:
test_eq(NBCell({}), NBCellRaw({}))
test_eq(NBCell(AD(cell_type='markdown')), {'cell_type': 'markdown', 'source': ''})
test_eq(
    NBCell(AD(cell_type='code', source='print("hello")')), 
    {'cell_type': 'code', 'source': 'print("hello")', 'outputs': []})
test_eq(
    NBCell(
        AD(cell_type='code', source='display("hello")', 
            outputs=[AD(output_type='display_data', data={'text/plain': 'hello'})])),
    {'cell_type': 'code', 'source': 'display("hello")', 'outputs': [AD(output_type='display_data', data={'text/plain': 'hello'})]})
test_eq(
    NBCell({
        'cell_type': 'code',
        'source': "display(HTML('cell 4'))\n",
        'outputs': ({
            'output_type': 'display_data',
            'data': {'text/html': 'cell 4', 'text/plain': '<IPython.core.display.HTML object>'},
            },)}),
    {'cell_type': 'code', 'source': "display(HTML('cell 4'))\n", 'outputs': [AD(output_type='display_data', data={'text/html': 'cell 4', 'text/plain': '<IPython.core.display.HTML object>'})]})

In [ ]:
c1 = NBCell(dict(cell_type='code', source='display("hello")'))
c2 = NBCell(c1)
test_is(c1, c2)

c3 = c2.copy()
test_is(c1 is c3, False)
test_eq(c1.source, c3.source)
test_eq(c1.outputs, c3.outputs)

In [ ]:
state['cells'][1]

{'idx': 1,
 'cell_type': 'code',
 'source': '# cell 1\nimport time\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()',
 'id': 'W1sZmlsZQ==',
 'metadata': {'brd': {'id': '717322f8-95fa-425c-839d-8b9e7d4ef921'}},
 'outputs': [],
 'execution_count': 1}

In [ ]:
NBCell(state['cells'][1])

{'idx': 1,
 'cell_type': 'code',
 'source': '# cell 1\nimport time\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()',
 'id': 'W1sZmlsZQ==',
 'metadata': {'brd': {'id': '717322f8-95fa-425c-839d-8b9e7d4ef921'}},
 'outputs': [],
 'execution_count': 1}

In [ ]:
cell = NBCell(state['cells'][2])
test_is(type(cell), NBCellCode)
test_eq_type(cell.outputs[0], NBOutput(AD(output_type='stream', name='stdout', text='1\n')))
cell

{'idx': 2,
 'cell_type': 'code',
 'source': '# cell 2\nprint(1)',
 'id': 'W2sZmlsZQ==',
 'metadata': {'brd': {'id': '92f3e304-fd32-4de2-badf-11577f9e7a4a'}},
 'outputs': [{'output_type': 'stream', 'name': 'stdout', 'text': '1\n'}],
 'execution_count': 2}

In [ ]:
cell = NBCell(state['cells'][4])
test_is(type(cell), NBCellCode)
cell

{'idx': 4,
 'cell_type': 'code',
 'source': "# cell 4\nimport time\ntime.sleep(2)\ndisplaydh(HTML('cell 4'), metadata={'bridge': {'cell': 4}});",
 'id': 'W4sZmlsZQ==',
 'metadata': {'brd': {'id': '798f7f83-9b31-4e21-a36b-5520c6f72a2d'}},
 'outputs': [{'output_type': 'display_data',
   'data': {'text/html': 'cell 4',
    'text/plain': '<IPython.core.display.HTML object>'},
   'metadata': {'transient': {'display_id': '2ffe60c5c571592fc61fe81faab39e34'},
    'bridge': {'cell': 4}}}],
 'execution_count': 4}

## did

In [ ]:
nested_idx(cell, 'outputs', 0, 'metadata', 'transient', 'display_id')

'2ffe60c5c571592fc61fe81faab39e34'

In [ ]:
test_eq(nested_idx(NBCell(state['cells'][0]), 'outputs', 0, 'metadata', 'transient', 'display_id'), None)

In [ ]:
nested_idx(getattr(NBCell(state['cells'][3]), 'outputs'), 0, 'metadata', 'transient', 'display_id')

'9d0548d3b88c66b7def8b47bbe5a12dd'

In [ ]:
cell = NBCell(state['cells'][8])
display(cell.outputs[0])
nested_idx(cell, 'outputs', 0, 'metadata', 'transient', 'display_id'), val_atpath(cell, 'outputs', 0, 'metadata', 'transient', 'display_id', default=None)

```json
{ 'data': { 'text/html': '<h3>cell 8</h3>\n',
            'text/plain': '<IPython.core.display.HTML object>'},
  'metadata': {'transient': {}},
  'output_type': 'display_data'}
```

({}, None)

can't use `nexsted_idx`.

In [ ]:
#| export

def did(o:NBOutput) -> str|None:
    if not (o and o['output_type'] == 'display_data'): return None
    md = o.metadata
    return md.get('brd_did') or val_atpath(md, 'transient', 'display_id', default=None) or None

FC.patch(did, as_prop=True)

In [ ]:
# type: ignore

cell = NBCell(state['cells'][4])
display(cell.outputs[0])

test_eq(did(cell.outputs[0]), cell.outputs[0]['metadata']['transient']['display_id'])
test_eq(cell.outputs[0].did, cell.outputs[0]['metadata']['transient']['display_id'])

```json
{ 'data': { 'text/html': 'cell 4',
            'text/plain': '<IPython.core.display.HTML object>'},
  'metadata': { 'bridge': {'cell': 4},
                'transient': { 'display_id': '2ffe60c5c571592fc61fe81faab39e34'}},
  'output_type': 'display_data'}
```

In [ ]:
cell = NBCell(state['cells'][2])
display(cell.outputs[0])
test_eq(did(cell.outputs[0]), None)

```json
{'name': 'stdout', 'output_type': 'stream', 'text': '1\n'}
```

In [ ]:
cell = NBCell(state['cells'][8])
display(cell.outputs[0])

```json
{ 'data': { 'text/html': '<h3>cell 8</h3>\n',
            'text/plain': '<IPython.core.display.HTML object>'},
  'metadata': {'transient': {}},
  'output_type': 'display_data'}
```

In [ ]:
cell = NBCell(state['cells'][8])
display(cell.outputs[0])
test_eq(did(cell.outputs[0]), None)

```json
{ 'data': { 'text/html': '<h3>cell 8</h3>\n',
            'text/plain': '<IPython.core.display.HTML object>'},
  'metadata': {'transient': {}},
  'output_type': 'display_data'}
```

## dids

In [ ]:
#| export

def dids(o:NBCell)->L[str]: return L(o.outputs if 'outputs' in o else () ).map(did).filter() # type: ignore

FC.patch(dids, as_prop=True)

In [ ]:
cell = NBCell(state['cells'][5])
display(cell.outputs)
test_eq(cell.dids, [cell.outputs[0].did])

[{'output_type': 'display_data',
  'data': {'application/javascript': 'console.log("cell 5")',
   'text/plain': '<IPython.core.display.Javascript object>'},
  'metadata': {'transient': {'display_id': 'b970be042e0ebd1e5af19aadcfbf213e'},
   'bridge': {'cell': 5}}}]

In [ ]:
test_eq(NBCell(state['cells'][1]).dids, [])

In [ ]:
display(state['cells'][2])
test_eq(NBCell(state['cells'][1]).dids, [])

{'idx': 2,
 'cell_type': 'code',
 'source': '# cell 2\nprint(1)',
 'id': 'W2sZmlsZQ==',
 'metadata': {'brd': {'id': '92f3e304-fd32-4de2-badf-11577f9e7a4a'}},
 'outputs': [{'output_type': 'stream', 'name': 'stdout', 'text': '1\n'}],
 'execution_count': 2}

In [ ]:
display(cell := NBCell(state['cells'][7]))
cell.dids

{'idx': 7,
 'cell_type': 'code',
 'source': "# cell 7\ndisplaydh(HTML('cell 7.1'), metadata={'bridge': {'cell': 7.1}})\ndisplaydh(HTML('cell 7.2'), metadata={'bridge': {'cell': 7.2}});",
 'id': 'X10sZmlsZQ==',
 'metadata': {'brd': {'id': '7512abfe-f7d2-4047-a6b1-56aefbf9457e'}},
 'outputs': [{'output_type': 'display_data',
   'data': {'text/html': 'cell 7.1',
    'text/plain': '<IPython.core.display.HTML object>'},
   'metadata': {'transient': {'display_id': '331d2582eb05fdc75afaa4c616fc8b6c'},
    'bridge': {'cell': 7.1}}},
  {'output_type': 'display_data',
   'data': {'text/html': 'cell 7.2',
    'text/plain': '<IPython.core.display.HTML object>'},
   'metadata': {'transient': {'display_id': 'bbb4397c26ce0c3fb2f1acddc6ce91b1'},
    'bridge': {'cell': 7.2}}}],
 'execution_count': 7}

(#2) ['331d2582eb05fdc75afaa4c616fc8b6c','bbb4397c26ce0c3fb2f1acddc6ce91b1']

In [ ]:
cells = L(state['cells']).map(NBCell)

In [ ]:
cells.map(lambda c: NBCell(c).dids)

(#42) [[],[],[],['9d0548d3b88c66b7def8b47bbe5a12dd'],['2ffe60c5c571592fc61fe81faab39e34'],['b970be042e0ebd1e5af19aadcfbf213e'],[],['331d2582eb05fdc75afaa4c616fc8b6c', 'bbb4397c26ce0c3fb2f1acddc6ce91b1'],[],[],[],[],[],[],[],[],['a6b5cc5f13bf64ce8dd73d0b79b0bef2'],['b0c4b491988399c7d88a82779528132d'],[],['e81dd25b299e4881aff6aec417c5bed9']...]

In [ ]:
test_eq(cells[2]['outputs'][0]['name'], 'stdout')  # type: ignore

In [ ]:
test_eq(len(codes := cells.argwhere(lambda c: c['cell_type'] == 'code')), 40)
display(codes)
cells[codes[2]]

(#40) [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20...]

{'idx': 3,
 'cell_type': 'code',
 'source': "# cell 3\ndisplaydh('cell 3', metadata={'bridge': {'cell': 3}});",
 'id': 'W3sZmlsZQ==',
 'metadata': {'brd': {'id': 'f2c19c18-a3f7-4acb-88e4-c7239178c401'}},
 'outputs': [{'output_type': 'display_data',
   'data': {'text/plain': "'cell 3'"},
   'metadata': {'transient': {'display_id': '9d0548d3b88c66b7def8b47bbe5a12dd'},
    'bridge': {'cell': 3}}}],
 'execution_count': 3}

## by_type

In [ ]:
#| export

def by_type(cells: Sequence[NBCell]|L, cell_type: CellTypesT) -> L:
    "Return 'L' of indices of cells of type `cell_type`"
    return L(cells).argwhere(lambda c: c['cell_type'] == cell_type)

In [ ]:

by_type(cells, 'code')

(#40) [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20...]

In [ ]:
by_type(cells, 'code')

(#40) [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20...]

## idx2cell

In [ ]:
idxs = by_type(cells, 'code')
idxs

(#40) [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20...]

In [ ]:
#| output: false

cells[idxs][:5]

(#5) [{'idx': 1, 'cell_type': 'code', 'source': '# cell 1\nimport time\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()', 'id': 'W1sZmlsZQ==', 'metadata': {'brd': {'id': '717322f8-95fa-425c-839d-8b9e7d4ef921'}}, 'outputs': [], 'execution_count': 1},{'idx': 2, 'cell_type': 'code', 'source': '# cell 2\nprint(1)', 'id': 'W2sZmlsZQ==', 'metadata': {'brd': {'id': '92f3e304-fd32-4de2-badf-11577f9e7a4a'}}, 'outputs': [{'output_type': 'stream', 'name': 'stdout', 'text': '1\n'}], 'execution_count': 2},{'idx': 3, 'cell_type': 'code', 'source': "# cell 3\ndisplaydh('cell 3', metadata={'bridge': {'cell': 3}});", 'id': 'W3sZmlsZQ==', 'metadata': {'brd': {'id': 'f2c19c18-a3f7-4acb-88e4-c7239178c401'}}, 'outputs': [{'output_type': 'display_data', 'data': {'text/plain': "'cell 3'"}, 'metadata': {'transient': {'display_i

In [ ]:
#| export

def idx2cell(cells: Sequence[NBCell]|L, cell_type: CellTypesT|None=None) -> Mapping[int, NBCell]:
    "Return mapping of indices to cells of type `cell_type`"
    cells = L(cells)
    idxs = by_type(cells, cell_type) if cell_type else L.range(len(cells))
    return dict(idxs.zipwith(cells[idxs]))

In [ ]:
test_eq(all(isinstance(c, NBCellCode) for c in idx2cell(cells, 'code').values()), True)
idx2cell(cells, 'code').keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41])

In [ ]:
idx2cell(cells, 'code')[1]

{'idx': 1,
 'cell_type': 'code',
 'source': '# cell 1\nimport time\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()',
 'id': 'W1sZmlsZQ==',
 'metadata': {'brd': {'id': '717322f8-95fa-425c-839d-8b9e7d4ef921'}},
 'outputs': [],
 'execution_count': 1}

## withOutputs

In [ ]:
outputs_idx = cells.argwhere(lambda c: 'outputs' in c)
display(outputs_idx)

print(outputs_idx.zipwith(cells[outputs_idx])[0])

dict(outputs_idx.zipwith(cells[outputs_idx]))[3]

(#40) [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20...]

(1, {'idx': 1, 'cell_type': 'code', 'source': '# cell 1\nimport time\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()', 'id': 'W1sZmlsZQ==', 'metadata': {'brd': {'id': '717322f8-95fa-425c-839d-8b9e7d4ef921'}}, 'outputs': [], 'execution_count': 1})


{'idx': 3,
 'cell_type': 'code',
 'source': "# cell 3\ndisplaydh('cell 3', metadata={'bridge': {'cell': 3}});",
 'id': 'W3sZmlsZQ==',
 'metadata': {'brd': {'id': 'f2c19c18-a3f7-4acb-88e4-c7239178c401'}},
 'outputs': [{'output_type': 'display_data',
   'data': {'text/plain': "'cell 3'"},
   'metadata': {'transient': {'display_id': '9d0548d3b88c66b7def8b47bbe5a12dd'},
    'bridge': {'cell': 3}}}],
 'execution_count': 3}

In [ ]:
#| export

def withOutputs(cells: Sequence[NBCell]|L):
    'Return indices of cells with outputs'
    return L(cells).argwhere(lambda c: 'outputs' in c)

In [ ]:
withOutputs(cells[:5])

(#4) [1,2,3,4]

## idx2outputs

In [ ]:
#| export

def idx2outputs(cells: Sequence[NBCell]|L) -> dict[int, L]:
    'Return dict of indices to cells with outputs'
    outputs_idx = withOutputs(cells := L(cells))
    return dict(outputs_idx.zipwith(cells[outputs_idx]))

In [ ]:
idx2outputs(cells)[2]

{'idx': 2,
 'cell_type': 'code',
 'source': '# cell 2\nprint(1)',
 'id': 'W2sZmlsZQ==',
 'metadata': {'brd': {'id': '92f3e304-fd32-4de2-badf-11577f9e7a4a'}},
 'outputs': [{'output_type': 'stream', 'name': 'stdout', 'text': '1\n'}],
 'execution_count': 2}

In [ ]:
idx2outputs(cells)[7]

{'idx': 7,
 'cell_type': 'code',
 'source': "# cell 7\ndisplaydh(HTML('cell 7.1'), metadata={'bridge': {'cell': 7.1}})\ndisplaydh(HTML('cell 7.2'), metadata={'bridge': {'cell': 7.2}});",
 'id': 'X10sZmlsZQ==',
 'metadata': {'brd': {'id': '7512abfe-f7d2-4047-a6b1-56aefbf9457e'}},
 'outputs': [{'output_type': 'display_data',
   'data': {'text/html': 'cell 7.1',
    'text/plain': '<IPython.core.display.HTML object>'},
   'metadata': {'transient': {'display_id': '331d2582eb05fdc75afaa4c616fc8b6c'},
    'bridge': {'cell': 7.1}}},
  {'output_type': 'display_data',
   'data': {'text/html': 'cell 7.2',
    'text/plain': '<IPython.core.display.HTML object>'},
   'metadata': {'transient': {'display_id': 'bbb4397c26ce0c3fb2f1acddc6ce91b1'},
    'bridge': {'cell': 7.2}}}],
 'execution_count': 7}

In [ ]:
idx2outputs(cells)[12]

{'idx': 12,
 'cell_type': 'code',
 'source': "# cell 12\nW.IntSlider(12, description='cell')",
 'id': 'X15sZmlsZQ==',
 'metadata': {'brd': {'id': 'e7e6d557-3aa7-41a7-9e77-805295112aec'}},
 'outputs': [{'output_type': 'execute_result',
   'data': {'application/vnd.jupyter.widget-view+json': {'version_major': 2,
     'version_minor': 0,
     'model_id': '6191b18d61a44fbd89eb132d22a28eab'},
    'text/plain': "IntSlider(value=12, description='cell')"},
   'execution_count': 12,
   'metadata': {}}],
 'execution_count': 12}

## idx2dids

In [ ]:
outputs_idx = withOutputs(cells := L(cells))
outputs_idx

(#40) [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20...]

In [ ]:
outputs_idx.zipwith(cells[outputs_idx]).filter(lambda c: c[1].dids)[:2]

(#2) [(3, {'idx': 3, 'cell_type': 'code', 'source': "# cell 3\ndisplaydh('cell 3', metadata={'bridge': {'cell': 3}});", 'id': 'W3sZmlsZQ==', 'metadata': {'brd': {'id': 'f2c19c18-a3f7-4acb-88e4-c7239178c401'}}, 'outputs': [{'output_type': 'display_data', 'data': {'text/plain': "'cell 3'"}, 'metadata': {'transient': {'display_id': '9d0548d3b88c66b7def8b47bbe5a12dd'}, 'bridge': {'cell': 3}}}], 'execution_count': 3}),(4, {'idx': 4, 'cell_type': 'code', 'source': "# cell 4\nimport time\ntime.sleep(2)\ndisplaydh(HTML('cell 4'), metadata={'bridge': {'cell': 4}});", 'id': 'W4sZmlsZQ==', 'metadata': {'brd': {'id': '798f7f83-9b31-4e21-a36b-5520c6f72a2d'}}, 'outputs': [{'output_type': 'display_data', 'data': {'text/html': 'cell 4', 'text/plain': '<IPython.core.display.HTML object>'}, 'metadata': {'transient': {'display_id': '2ffe60c5c571592fc61fe81faab39e34'}, 'bridge': {'cell': 4}}}], 'execution_count': 4})]

In [ ]:
#| export

def idx2dids(cells: Sequence[NBCell]|L) -> dict[int, L]:
    'Return dict of indices to cells with transient outputs'
    idxs = withOutputs(cells := L(cells))
    return dict(idxs.zipwith(cells[idxs].map(dids)).filter(lambda c: c[1]))  # type: ignore

In [ ]:
idx2dids(cells).keys(), idx2dids(cells)[7]

(dict_keys([3, 4, 5, 7, 16, 17, 19, 24, 26, 35]),
 (#2) ['331d2582eb05fdc75afaa4c616fc8b6c','bbb4397c26ce0c3fb2f1acddc6ce91b1'])

## directives
> `NBCell`s register **nbdev [directives](https://nbdev.fast.ai/explanations/directives.html)**/**quarto [Cell options](https://quarto.org/docs/reference/cells/cells-jupyter.html)**/**python [cell magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cell-magics)** in `directives_`.

In [ ]:
s = '''
%%python
#| hide

print(1)
'''
cell1 = NBCell(AD(cell_type='code', source=s))
test_is(cell1.has_directive('hide'), True)
cell1.directives_

{'python': [], 'hide': []}

In [ ]:
s = '''
#| label: fig-polar
#| echo: false

# comment
print(1)
'''
cell2 = NBCell(dict(cell_type='code', source=s))
cell2.directives_

{'label:': ['fig-polar'], 'echo:': ['false']}

In [ ]:
s = '''

    #| code-fold

print(1)
'''
cell3 = NBCell({'cell_type':'code', 'source':s})
cell3.directives_

{'code-fold': []}

In [ ]:
#| export

def by_directive(cells: Sequence[NBCell]|L, directive: str, *args):
    return L(cells).argwhere(lambda c: has_directive(c, directive, *args))

In [ ]:
test_eq(by_directive([cell1, cell2, cell3], 'hide'), [0])

# NB

In [ ]:
{'a', 'b'} & {'a', 'c'}

{'a'}

In [ ]:
#| export

JSONVal: TypeAlias = str | int | float | bool | None | dict[str, 'JSONVal'] | list['JSONVal']

@runtime_checkable
class NBProvider(Protocol):
    "Objects that provide access to a notebook (NB) instance"
    @property
    def nb(self) -> NB: ...

@runtime_checkable
class NBProcessor(Protocol):
    "Callable that transforms a notebook and returns the result"
    def __call__(self, nb:NB, *args, **kwargs) -> NB: ...


class NB:
    "Bridget representation of notebook state"
    cells: L[NBCell]  # type: ignore
    nbData: dict
    type: str
    timestamp: str
    origin: str
    def __init__(self, cells: Sequence[NBCell]|L=(), **kwargs): self.setup(cells, **kwargs)
    def setup(self, cells: Sequence[NBCell]|L=(), **kwargs):
        self.cells = L(NBCell(_) for _ in cells)
        d = update_({'nbData':{}, 'type':'state', 'timestamp':'', 'origin':''}, **_relevant_kw(self, kwargs))
        for k,v in d.items(): setattr(self, k, v)
        self._rebuild()
    @classmethod
    def fromStateMessage(cls, message): return cls(**message)
    @classmethod
    def from_NB(cls, nb: NB, cells: Sequence[NBCell]|L|None=None, **kwargs): 
        return NB(cells if cells is not None else nb.cells, 
			**update_({'nbData':nb.nbData, 'type':nb.type, 'timestamp':nb.timestamp}, **kwargs))
    def as_dict(self): return _relevant_kw(self, vars(self)) 

    def _indxs(self):
        self._id2idx = {c.id:i for i,c in enumerate(self.cells)}  # type: ignore
    
    def __iter__(self): return iter(self.cells)
    @overload
    def __getitem__(self, key: SupportsIndex|str, /) -> NBCell: ...
    @overload
    def __getitem__(self, key: slice, /) -> L: ...
    def __getitem__(self, key) -> NBCell|L: 
        if isinstance(key, str):
            if (idx := self._id2idx.get(key)) is not None: return self.cells[idx]  # type: ignore
            # cc = self.cells.filter(lambda c: c.id == key)
            # if cc: return cc[0] if len(cc) else L()  # type: ignore
            cc = self.cells.filter(lambda c: 'metadata' in c and c.metadata['cell_id'] == key)
            return cc[0] if len(cc) else L()  # type: ignore
        return self.cells[key]  # type: ignore
    
    def by_type(self, cell_type: CellTypesT): return by_type(self.cells, cell_type)
    @cached_property
    def codes(self): return by_type(self.cells, 'code')
    @cached_property
    def mds(self): return by_type(self.cells, 'markdown')

    def idx2cell(self, cell_type: CellTypesT|None=None): return idx2cell(self.cells, cell_type)
    @cached_property
    def idx2code(self): return idx2cell(self.cells, 'code')
    @cached_property
    def idx2md(self): return idx2cell(self.cells, 'markdown')

    @cached_property
    def withOutputs(self): return withOutputs(self.cells)
    @cached_property
    def idx2outputs(self) -> dict[int, L]: return idx2outputs(self.cells)
    @cached_property
    def idx2dids(self) -> dict[int, L]: return idx2dids(self.cells)

    def by_directive(self, directive: str, *args): return by_directive(self.cells, directive)
    @cached_property
    def hiddens(self): return self.by_directive('hide') + self.by_directive('include', 'false')

    @cache
    def cell_by_did(self, did:str) -> NBCell|None:
        cc = self.cells[tuple(self.idx2dids.keys())].filter(lambda c: did in dids(c))  # type: ignore
        return cc[0] if len(cc) else None  # type: ignore

    @cache
    def select(self, k='source'): return self.cells.attrgot(k, None)

    _cks = {'codes', 'mds', 'idx2code', 'idx2md', 'withOutputs', 'idx2outputs', 'idx2dids', 'hiddens'}
    def _rebuild(self):
        self.select.cache_clear()
        self.cell_by_did.cache_clear()
        self._indxs()
        vv = vars(self).keys() & self._cks
        for k in vv:delattr(self, k)

    def _process(self, items, cbs):
        collections.deque(CollBack(items, context=self, cbs=cbs), maxlen=0)
        return FC.first(cbs.attrgot('nb')[::-1]) or self
    
    def process(self, /,
            cbs: Callback|Sequence[Callback]=(), 
            slc:slice|None=None, 
            pred:Callable[[NBCell], bool]|None=None, 
            **kwargs  # FuncCB kwargs
        ) -> NB:
        "Process a subset `slc` of cells filtered by `pred` with `cbs` and `FuncCB` callbacks."
        _cbs = FC.L(FC.tuplify(cbs) + ((FuncCB(**kwargs),) if kwargs else ()))
        items = self.cells[slc or slice(None)].filter(pred) if pred or slc else self.cells  # type: ignore
        if _cbs: return self._process(items, _cbs)
        else: return NB(items, nbData=self.nbData, type=self.type, timestamp=self.timestamp)
    
    def pipe(self, funcs:Iterable[NBProcessor], *args, **kwargs) -> NB: 
        return compose_first(*funcs)(self, *args, **kwargs)  # type: ignore
    
    # hate but a lot wiggly reds but mostly hate pyright's hatred of notebooks
    def _apply_diff(self, diff:dict): ...
    def apply_diffsMessage(self, diffs): ...
    def find(self, what:JSONVal|Callable[[NBCell], bool], where:str='source', op=op.contains) -> L: ...
    def found(self, 
        what:JSONVal|Callable[[NBCell], bool], where='source', op=op.contains, 
        cbs:Callback|Sequence[Callback]=()
    ) -> NB: ...
    @property
    def source(self) -> L: ...
    @property
    def metadata(self) -> L: ...
    @property
    def outputs(self) -> L: ...


for k in ('source', 'metadata', 'outputs'): setattr(NB, k, property(NB.select.__wrapped__,))

In [ ]:
test_eq(NB().as_dict(), {'cells': [], 'nbData': {}, 'type': 'state', 'timestamp': '', 'origin': ''})
test_eq(NB(nbData={'a':1}).as_dict(), {'cells': [], 'nbData':{'a':1}, 'type': 'state', 'timestamp': '', 'origin': ''})

In [ ]:
nb = NB.fromStateMessage(state)
cells = nb.cells

test_eq(len(nb.cells), nb.nbData['cellCount'])
test_eq(nb.nbData['metadata']['nbformat'], 4)
(c := nb.cells[2])

{'idx': 2,
 'cell_type': 'code',
 'source': '# cell 2\nprint(1)',
 'id': 'W2sZmlsZQ==',
 'metadata': {'brd': {'id': '92f3e304-fd32-4de2-badf-11577f9e7a4a'}},
 'outputs': [{'output_type': 'stream', 'name': 'stdout', 'text': '1\n'}],
 'execution_count': 2}

In [ ]:
test_eq(c, nb[2])
test_eq(nb[c.id], nb[2])  # type: ignore

In [ ]:
nb[12]

{'idx': 12,
 'cell_type': 'code',
 'source': "# cell 12\nW.IntSlider(12, description='cell')",
 'id': 'X15sZmlsZQ==',
 'metadata': {'brd': {'id': 'e7e6d557-3aa7-41a7-9e77-805295112aec'}},
 'outputs': [{'output_type': 'execute_result',
   'data': {'application/vnd.jupyter.widget-view+json': {'version_major': 2,
     'version_minor': 0,
     'model_id': '6191b18d61a44fbd89eb132d22a28eab'},
    'text/plain': "IntSlider(value=12, description='cell')"},
   'execution_count': 12,
   'metadata': {}}],
 'execution_count': 12}

In [ ]:
#|export

add_docs(NB,
    setup="Setup with new data",
    as_dict="Return dict of all attributes",
    by_type="Return indices of cells of type `cell_type`",
    codes="Cell indices of type `code`",
    mds="Cell indices of type `markdown`",
    idx2cell="Return dict of indices to cells of type `cell_type`",
    idx2code="Return dict of indices to cells of type `code`",
    idx2md="Return dict of indices to cells of type `markdown`",
    withOutputs="Return indices of cells with outputs",
    idx2outputs="Return dict of indices to cells with outputs",
    by_directive="Return indices of cells with directive `directive`",
    hiddens="Return indices of cells with `hide` or `include:false` directives",
    pipe="Pipe a sequence of `NBProcessor`s",
    from_NB="Create a new `NB` from an existing `NB`",
    cell_by_did="Return cell with `did` or None",
    select="Return `k` attribute of cells",
    source="Return `source` attribute of all cells",
    metadata="Return `metadata` attribute of all cells",
    outputs="Return `outputs` attribute of all cells",
    apply_diffsMessage="Apply a diffsMessage to the `NB`",
    find="Find cells matching `what` in `where` using `op`",
    found="Find cells matching `what` in `where` using `op` and process them with `cbs`",
)

In [ ]:
show_doc(NB.by_type)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L238){target="_blank" style="float:right; font-size:smaller"}

### NB.by_type

>      NB.by_type (cell_type:Literal['code','markdown'])

*Return indices of cells of type `cell_type`*

In [ ]:
display(nb[2])
test_eq(cells[2].outputs[0].name, 'stdout')  # type: ignore

{'idx': 2,
 'cell_type': 'code',
 'source': '# cell 2\nprint(1)',
 'id': 'W2sZmlsZQ==',
 'metadata': {'brd': {'id': '92f3e304-fd32-4de2-badf-11577f9e7a4a'}},
 'outputs': [{'output_type': 'stream', 'name': 'stdout', 'text': '1\n'}],
 'execution_count': 2}

In [ ]:
nb.by_type('code')

(#40) [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20...]

In [ ]:
show_doc(NB.codes)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L240){target="_blank" style="float:right; font-size:smaller"}

### NB.codes



*Cell indices of type `code`*

In [ ]:
show_doc(NB.mds)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L242){target="_blank" style="float:right; font-size:smaller"}

### NB.mds



*Cell indices of type `markdown`*

In [ ]:
nb.mds

(#2) [0,23]

In [ ]:
show_doc(NB.idx2cell)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L244){target="_blank" style="float:right; font-size:smaller"}

### NB.idx2cell

>      NB.idx2cell (cell_type:Optional[Literal['code','markdown']]=None)

*Return dict of indices to cells of type `cell_type`*

In [ ]:
nb.idx2cell('code')[2]

{'idx': 2,
 'cell_type': 'code',
 'source': '# cell 2\nprint(1)',
 'id': 'W2sZmlsZQ==',
 'metadata': {'brd': {'id': '92f3e304-fd32-4de2-badf-11577f9e7a4a'}},
 'outputs': [{'output_type': 'stream', 'name': 'stdout', 'text': '1\n'}],
 'execution_count': 2}

In [ ]:
show_doc(NB.idx2code)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L246){target="_blank" style="float:right; font-size:smaller"}

### NB.idx2code



*Return dict of indices to cells of type `code`*

In [ ]:
show_doc(NB.idx2md)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L248){target="_blank" style="float:right; font-size:smaller"}

### NB.idx2md



*Return dict of indices to cells of type `markdown`*

In [ ]:
nb.idx2md[0]

{'idx': 0,
 'cell_type': 'markdown',
 'source': '# cell 0',
 'id': 'W0sZmlsZQ==',
 'metadata': {'brd': {'id': '81c3e877-b59a-48f5-a1da-21894d928d4b'}}}

In [ ]:
show_doc(NB.withOutputs)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L251){target="_blank" style="float:right; font-size:smaller"}

### NB.withOutputs



*Return indices of cells with outputs*

In [ ]:
show_doc(NB.idx2outputs)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L253){target="_blank" style="float:right; font-size:smaller"}

### NB.idx2outputs



*Return dict of indices to cells with outputs*

In [ ]:
# type: ignore

display(cells[codes[2]].outputs[0])

test_eq(did(cells[codes[2]].outputs[0]), cells[codes[2]].outputs[0].metadata['transient']['display_id'])
test_eq(cells[codes[2]].outputs[0].did, cells[codes[2]].outputs[0].metadata['transient']['display_id'])

```json
{ 'data': {'text/plain': "'cell 3'"},
  'metadata': { 'bridge': {'cell': 3},
                'transient': { 'display_id': '9d0548d3b88c66b7def8b47bbe5a12dd'}},
  'output_type': 'display_data'}
```

In [ ]:
print(nb.idx2outputs.keys())
display(nb.idx2outputs[1])
nb.idx2outputs[2]

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41])


{'idx': 1,
 'cell_type': 'code',
 'source': '# cell 1\nimport time\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()',
 'id': 'W1sZmlsZQ==',
 'metadata': {'brd': {'id': '717322f8-95fa-425c-839d-8b9e7d4ef921'}},
 'outputs': [],
 'execution_count': 1}

{'idx': 2,
 'cell_type': 'code',
 'source': '# cell 2\nprint(1)',
 'id': 'W2sZmlsZQ==',
 'metadata': {'brd': {'id': '92f3e304-fd32-4de2-badf-11577f9e7a4a'}},
 'outputs': [{'output_type': 'stream', 'name': 'stdout', 'text': '1\n'}],
 'execution_count': 2}

In [ ]:
(cell := nb.idx2outputs[7])

{'idx': 7,
 'cell_type': 'code',
 'source': "# cell 7\ndisplaydh(HTML('cell 7.1'), metadata={'bridge': {'cell': 7.1}})\ndisplaydh(HTML('cell 7.2'), metadata={'bridge': {'cell': 7.2}});",
 'id': 'X10sZmlsZQ==',
 'metadata': {'brd': {'id': '7512abfe-f7d2-4047-a6b1-56aefbf9457e'}},
 'outputs': [{'output_type': 'display_data',
   'data': {'text/html': 'cell 7.1',
    'text/plain': '<IPython.core.display.HTML object>'},
   'metadata': {'transient': {'display_id': '331d2582eb05fdc75afaa4c616fc8b6c'},
    'bridge': {'cell': 7.1}}},
  {'output_type': 'display_data',
   'data': {'text/html': 'cell 7.2',
    'text/plain': '<IPython.core.display.HTML object>'},
   'metadata': {'transient': {'display_id': 'bbb4397c26ce0c3fb2f1acddc6ce91b1'},
    'bridge': {'cell': 7.2}}}],
 'execution_count': 7}

In [ ]:
def get_data(cell: NBCell, mime:str|None=None): 
    return d.get(mime, None) if (d := nested_idx(cell, 'outputs', 0, 'data')) and mime else d
get_html = partial(get_data, mime='text/html')
get_plain = partial(get_data, mime='text/plain')

print(f"{get_data(cell)=}\n{get_html(cell)=}\n{get_plain(cell)=}")

get_data(cell)={'text/html': 'cell 7.1', 'text/plain': '<IPython.core.display.HTML object>'}
get_html(cell)='cell 7.1'
get_plain(cell)='<IPython.core.display.HTML object>'


In [ ]:
show_doc(NB.process)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L281){target="_blank" style="float:right; font-size:smaller"}

### NB.process

>      NB.process
>                  (cbs:Union[pote.callback.Callback,Sequence[pote.callback.Call
>                  back]]=(), slc:slice|None=None,
>                  pred:Optional[Callable[[__main__.NBCell],bool]]=None,
>                  **kwargs)

*Process a subset `slc` of cells filtered by `pred` with `cbs` and `FuncCB` callbacks.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| cbs | Union | () |  |
| slc | slice \| None | None |  |
| pred | Optional | None |  |
| kwargs | VAR_KEYWORD |  |  |
| **Returns** | **NB** |  | **FuncCB kwargs** |

In [ ]:
nb.process(on_iter=lambda _,item: print(item.cell_type, end=', '));

markdown, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, markdown, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, code, 

In [ ]:
ctyps = []
cb = FuncCB(on_iter=lambda _,item: ctyps.append(item.cell_type))
process_(nb.cells, cb)
test_eq(ctyps, [c.cell_type for c in nb.cells])

ctyps = []
nb.process(slc=slice(0,2), on_iter=lambda _,item: ctyps.append(item.cell_type))
test_eq(ctyps, ('markdown', 'code'))

In [ ]:
nb.process(on_iter=lambda _,item: print((_.n, item.cell_type), end=', '));

(0, 'markdown'), (1, 'code'), (2, 'code'), (3, 'code'), (4, 'code'), (5, 'code'), (6, 'code'), (7, 'code'), (8, 'code'), (9, 'code'), (10, 'code'), (11, 'code'), (12, 'code'), (13, 'code'), (14, 'code'), (15, 'code'), (16, 'code'), (17, 'code'), (18, 'code'), (19, 'code'), (20, 'code'), (21, 'code'), (22, 'code'), (23, 'markdown'), (24, 'code'), (25, 'code'), (26, 'code'), (27, 'code'), (28, 'code'), (29, 'code'), (30, 'code'), (31, 'code'), (32, 'code'), (33, 'code'), (34, 'code'), (35, 'code'), (36, 'code'), (37, 'code'), (38, 'code'), (39, 'code'), (40, 'code'), (41, 'code'), 

In [ ]:
def _enumerate(istat, item:NBCell): 
    item.metadata['n'] = istat.n

test_eq(vals_at(nb.cells, '*.metadata.n'), ())
nb.process(on_iter=_enumerate);
test_eq(vals_at(nb.cells, '*.metadata.n'), range(len(nb.cells)))

In [ ]:
show_doc(NB.pipe)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L293){target="_blank" style="float:right; font-size:smaller"}

### NB.pipe

>      NB.pipe (funcs:Iterable[__main__.NBProcessor], *args, **kwargs)

*Pipe a sequence of `NBProcessor`s*

In [ ]:
nb.pipe((lambda nb: NB.from_NB(nb, nb.cells[nb.mds]),)).cells.attrgot('source')

(#2) ['# cell 0','# cell 23']

In [ ]:
show_doc(NB.cell_by_did)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/nb.py#L262){target="_blank" style="float:right; font-size:smaller"}

### NB.cell_by_did

>      NB.cell_by_did (did:str)

In [ ]:
test_eq(nb.cell_by_did(nb[7].dids[1]), nb[7])

# Diffs

In [ ]:
def check(self:NB):
    test_eq(self.nbData['cellCount'], len(self.cells))
    for n, c in enumerate(self.cells):
        test_eq(c.idx, n)

check(nb)

In [ ]:
state = json.loads(Path(test_fn).read_text('utf-8'))
nb = NB.fromStateMessage(state)
test_eq(len(nb.cells), nb.nbData['cellCount'])

In [ ]:
#| output: false

# update existing cell
diffs1 = json.loads(Path('../packages/nbinspect-vscode/test/update01_chg_2.json').read_text('utf-8'))
RenderJSON(diffs1, init_level=4, max_height=400).display()

In [ ]:
def apply_diff(nb:NB, diff):
    changed, added, removed, cellCount = gets(diff, 'cells', 'added', 'removed', 'cellCount')
    cells = nb.cells
    for cell in changed:  # type: ignore
        cell = NBCell(cell)
        cells[cell.idx] = cell

apply_diff(nb, diffs1['changes'][0])
check(nb)

In [ ]:
#| output: false

# remove cell
diffs2 = json.loads(Path('../packages/nbinspect-vscode/test/update13_remove_2-3-5.json').read_text('utf-8'))
RenderJSON(diffs2, init_level=5, max_height=400).display()

In [ ]:
def apply_diff(nb:NB, diff):
    changed, added, removed, cellCount = gets(diff, 'cells', 'added', 'removed', 'cellCount')
    cells = nb.cells
    if removed:
        for idx in sorted(removed, reverse=True):  # type: ignore
            del cells[idx]
            for cell in cells[idx:]:
                cell.idx -= 1
    nb.nbData['cellCount'] = cellCount
    for cell in changed:  # type: ignore
        cells[cell.idx] = NBCell(cell)

nb = NB.fromStateMessage(state)
apply_diff(nb, diffs2['changes'][0])
apply_diff(nb, diffs2['changes'][1])
check(nb)

In [ ]:
#| export

@FC.patch
def _apply_diff(self:NB, diff:dict):
    changed, added, removed, cell_count = gets(diff, 'cells', 'added', 'removed', 'cellCount')
    final_count = len(self.cells) - len(removed if isinstance(removed, list) else []) + len(added if isinstance(added, list) else [])
    if cell_count != final_count:
        raise IndexError(f"---- inconsistent diff cell_count: {cell_count} != {final_count=}")
    cells = self.cells
    if isinstance(removed, list) and removed:
        if (n := max(removed)) >= len(cells):
            raise IndexError(f"---- inconsistent diff removed: {n} > {len(cells)}")
        for idx in sorted(removed, reverse=True):
            del cells[idx]
            for cell in cells[idx:]: cell.idx -= 1
    elif isinstance(added, list) and added:
        for cell in added:
            if (idx := cell['idx']) > len(cells):
                raise IndexError(f"---- inconsistent diff added: {idx} > {len(cells)=}")
            cells.insert(idx, NBCell(cell))
            for cell in cells[idx+1:]: cell.idx += 1
    ln = len(cells)
    if isinstance(changed, list):
        for cell in changed: 
            if (idx := cell['idx']) > ln:
                raise IndexError(f"---- inconsistent diff changed: {idx} > {len(cells)=}")
            cells[idx] = NBCell(cell)
    self.nbData['cellCount'] = cell_count
    for n, c in enumerate(self.cells):
        if c.idx != n:
            raise IndexError(f"---- inconsistent diff cell idx: {c.idx} != {n=}")
    self._rebuild()

In [ ]:
#| output: false

# cells added
diffs3 = json.loads(Path('../packages/nbinspect-vscode/test/update14-add_2-3-4.json').read_text('utf-8'))
RenderJSON(diffs3, init_level=4, max_height=400).display()

In [ ]:
nb = NB.fromStateMessage(state)

nb._apply_diff(diffs1['changes'][0])
nb._apply_diff(diffs2['changes'][0])
nb._apply_diff(diffs2['changes'][1])
nb._apply_diff(diffs3['changes'][0])
check(nb)

In [ ]:
# type: ignore
nb = NB.fromStateMessage(state)

nb._apply_diff({'cells': [], 'added': [], 'removed': [], 'cellCount': 42})
check(nb)

with test_raises(IndexError):
    nb._apply_diff({'cells': [], 'added': [], 'removed': [], 'cellCount': 134})

nb._apply_diff({'cells': [], 'added': [], 'removed': [3, 5], 'cellCount': 40})
check(nb)

with test_raises(IndexError):
    nb._apply_diff({'cells': [], 'added': [], 'removed': [3, 5], 'cellCount': 40})

with test_raises(IndexError):
    nb._apply_diff({'added': [{'idx': 50}], 'cells': [], 'removed': [], 'cellCount': 41})

with test_raises(IndexError):
    nb._apply_diff({'removed': [43], 'cells': [], 'added': [], 'cellCount': 40})

with test_raises(IndexError):
    nb._apply_diff({'cells': [{'idx': 41}], 'added': [], 'removed': [], 'cellCount': 40})

In [ ]:
#| export

@FC.patch
def apply_diffsMessage(self:NB, diffs):
    for diff in diffs['changes']: 
        try: self._apply_diff(diff)  # type: ignore
        except IndexError as e:
            print(f"---- inconsistent diff: {e}")
            raise e

In [ ]:
nb = NB.fromStateMessage(state)

ff = sorted(Path('../packages/nbinspect-vscode/test/').glob('update*.json'))
for f in ff[:12]:
    diffs = json.loads(f.read_text('utf-8'))
    nb.apply_diffsMessage(diffs)
check(nb)

In [ ]:
nb = NB.fromStateMessage(state)

for f in ff[12:]:
    diffs = json.loads(f.read_text('utf-8'))
    nb.apply_diffsMessage(diffs)
check(nb)

# IpynbConvertCB
> Notebook state to IPython notebook format (ipynb)

NBCell and friends are mostly [Notebook format types](https://nbformat.readthedocs.io/en/4.4.0/format_description.html#cell-types) without enforcing some attributes. Going back and forth between the two formats is easy.

In [ ]:
#| export

class IpynbOutput(AD): ...
class IpynbCell(AD): outputs: list[IpynbOutput]

class St2Ipynb(Transform): 
    def __init__(self, in_vscode: bool=False): self.in_vscode = in_vscode
    def encodes(self, x: NBOutput) -> IpynbOutput:  # type: ignore
        fmt = IpynbOutput(x)
        if fmt['output_type'] in ('display_data', 'execute_result'): fmt.metadata=x.get('metadata', {})
        fmt.get('metadata', emptyd).pop('transient', None)
        return fmt
    def decodes(self, x: IpynbOutput) -> NBOutput: return NBOutput(x)  # type: ignore
        

    def encodes(self, x: NBCell) -> IpynbCell:
        fmt = update_(IpynbCell(x), metadata=x.get('metadata', {}))
        if self.in_vscode:
            fmt['metadata'] = deepcopy(fmt['metadata'])
            brd = fmt['metadata']['brd']
            brd['cell_id'], fmt['id'] = fmt['id'], brd['id']
            del brd['id']
        fmt.pop('idx', None)
        if 'outputs' in x: fmt['outputs'] = self(tuple(x.outputs))
        # if x['cell_type'] == 'code': 
        #     fmt['execution_count'] = fmt.get('execution_count', None)
        return fmt
    def decodes(self, x: IpynbCell) -> NBCell: return NBCell(x)

In [ ]:
cell = NBCell({
    'cell_type': 'code', 'source': '', 'idx': 1, 'id': 'asdfg',
    'metadata': {'brd': {'id': '123-456'}},
    'outputs': [{
        'output_type': 'display_data',
        'data': {'text/plain': ''},
        'metadata': {'transient': {'display_id': 'abc'}}
    }]
})
c = St2Ipynb(in_vscode)(cell)
test_eq(type(c), IpynbCell)
test_eq(c.outputs[0].metadata, {})
test_eq(c.id, '123-456' if in_vscode else 'asdfg')
test_fail(lambda: c.idx)

In [ ]:
#| output: false

nb = NB.fromStateMessage(state)

icells = nb.cells.map(St2Ipynb(in_vscode))
RenderJSON(icells, init_level=1, max_height=400).display()

In [ ]:
#| export

class IpynbConvertCB(Callback):
    def __init__(self, in_vscode: bool=False): self.in_vscode = in_vscode
    def before_iter(self, istat): 
        nb = istat.context
        d, ks, mdks = nb.nbData['metadata'], ('nbformat', 'nbformat_minor'), ('kernelspec', 'language_info')
        self.notebook = {
            'cells': [],
            **FC.filter_keys(d, FC.in_(ks)),  # type: ignore
            'metadata': FC.filter_keys(d['metadata'], FC.in_(mdks)),  # type: ignore
        }
        self.notebook['nbformat_minor'] = 5
        self.cell_counter = 0
        self._fmt = St2Ipynb(self.in_vscode)
    
    def on_iter(self, _, cell):
        # fmt = update_(St2Ipynb()(cell), id=self.cell_counter)
        self.notebook['cells'].append(self._fmt(cell))
        self.cell_counter += 1

In [ ]:
#| output: false

# processor = BridgetNBProcessor(state)
nb.process(cb := IpynbConvertCB(in_vscode))
ipynb_json = cb.notebook

nb = nbformat.reads(json.dumps(ipynb_json), as_version=nbformat.NO_CONVERT, capture_validation_error=(derr := {}))
test_eq(derr, {})

RenderJSON(ipynb_json, init_level=1, max_height=400).display()

# MDConvertCB


# StateConversionCB

In [ ]:
#| hide

# class StateConversionCB(Callback):
#     @cached_property
#     def state(self) -> AD: return AD(type='state',cells=[])
    
#     def _convert_output(self, output):
#         converted = dict(output)
#         if 'output_type' in converted:
#             converted['metadata']['outputType'] = converted.pop('output_type')
#         return converted
    
#     def on_iter(self, ctx, cell):
#         # Convert each cell back to the original state format
#         st_cell = NBCell({
#             'cell_type': cell.cell_type,
#             'source': ''.join(cell.source) if isinstance(cell.source, list) else cell.source,
#         })
#         if cell.metadata: st_cell.metadata = cell.metadata
#         if cell.cell_type == 'code':
#             st_cell['outputs'] = [self._convert_output(o) for o in cell.outputs]
#         self.state.cells.append(st_cell)

# tracker = CollBack(nb.cells, cbs=[cb := StateConversionCB()])
# tracker.update(item=nb.cells[0])
# test_eq(cell, nb.cells[0])

# # with tracker.this_cbs([cb := StateConversionCB()]):
# #     # for cell in tracker: pass
# #     cell = next(iter(tracker))
# # converted_state = cb.state

# # # Compare with original state
# # for i, c1,c2 in zip(range(len(converted_state.cells)), converted_state.cells, AD(state).cells):
# #     cprint(c1, c2)
# #     test_eq(c1, c2)
# # # assert converted_state['cells'] == state['cells'], "Round-trip conversion failed"

# Find

In [ ]:
class FindCB(Callback):
    def __init__(self, what:str|Callable[[NBCell], bool], where:str='source'): 
        self.what = (lambda c: what in getattr(c, where)) if isinstance(what, str) else what
        self._cells = []
    @property
    def nb(self): return NB(self._cells)
    def on_iter(self, _, cell):
        if has_directive(cell, 'hide'): return
        if self.what(cell): self._cells.append(cell)

In [ ]:
nb = NB.fromStateMessage(state)

In [ ]:
cb = FindCB('displaydh')
[*shortens(nb.process(cb).cells, 'r', 80)]

["{'idx': 1, 'cell_type': 'code', 'source': '# cell 1\\nimport time\\nfrom itertools…",
 '{\'idx\': 3, \'cell_type\': \'code\', \'source\': "# cell 3\\ndisplaydh(\'cell 3\', metadat…',
 '{\'idx\': 4, \'cell_type\': \'code\', \'source\': "# cell 4\\nimport time\\ntime.sleep(2)\\…',
 "{'idx': 5, 'cell_type': 'code', 'source': '# cell 5\\ndisplaydh(Javascript(\\'cons…",
 '{\'idx\': 7, \'cell_type\': \'code\', \'source\': "# cell 7\\ndisplaydh(HTML(\'cell 7.1\'),…',
 '{\'idx\': 16, \'cell_type\': \'code\', \'source\': "# cell 16\\ndh = displaydh(display_id…',
 '{\'idx\': 17, \'cell_type\': \'code\', \'source\': "# cell 17\\ndh = displaydh(HTML(\'cell…',
 '{\'idx\': 19, \'cell_type\': \'code\', \'source\': "# cell 19\\ndisplaydh(JSON({\'cell\': 1…',
 '{\'idx\': 24, \'cell_type\': \'code\', \'source\': "# cell 24\\nfrom itertools import cou…',
 '{\'idx\': 26, \'cell_type\': \'code\', \'source\': "# cell 26\\n_ = displaydh(\'cell 26.1\'…',
 '{\'idx\': 35, \'cell_type\': \'code\', \'source\': 

In [ ]:
list(nb.cells.filter(lambda c: 'displaydh' in c.source).map(shorten, mode='r', limit=80))

["{'idx': 1, 'cell_type': 'code', 'source': '# cell 1\\nimport time\\nfrom itertools…",
 '{\'idx\': 3, \'cell_type\': \'code\', \'source\': "# cell 3\\ndisplaydh(\'cell 3\', metadat…',
 '{\'idx\': 4, \'cell_type\': \'code\', \'source\': "# cell 4\\nimport time\\ntime.sleep(2)\\…',
 "{'idx': 5, 'cell_type': 'code', 'source': '# cell 5\\ndisplaydh(Javascript(\\'cons…",
 '{\'idx\': 7, \'cell_type\': \'code\', \'source\': "# cell 7\\ndisplaydh(HTML(\'cell 7.1\'),…',
 '{\'idx\': 16, \'cell_type\': \'code\', \'source\': "# cell 16\\ndh = displaydh(display_id…',
 '{\'idx\': 17, \'cell_type\': \'code\', \'source\': "# cell 17\\ndh = displaydh(HTML(\'cell…',
 '{\'idx\': 19, \'cell_type\': \'code\', \'source\': "# cell 19\\ndisplaydh(JSON({\'cell\': 1…',
 '{\'idx\': 24, \'cell_type\': \'code\', \'source\': "# cell 24\\nfrom itertools import cou…',
 '{\'idx\': 26, \'cell_type\': \'code\', \'source\': "# cell 26\\n_ = displaydh(\'cell 26.1\'…',
 '{\'idx\': 35, \'cell_type\': \'code\', \'source\': 

In [ ]:
list(nb.source.filter(lambda c: 'displaydh' in c))  # type: ignore

['# cell 1\nimport time\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()',
 "# cell 3\ndisplaydh('cell 3', metadata={'bridge': {'cell': 3}});",
 "# cell 4\nimport time\ntime.sleep(2)\ndisplaydh(HTML('cell 4'), metadata={'bridge': {'cell': 4}});",
 '# cell 5\ndisplaydh(Javascript(\'console.log("cell 5")\'), metadata={\'bridge\': {\'cell\': 5}});',
 "# cell 7\ndisplaydh(HTML('cell 7.1'), metadata={'bridge': {'cell': 7.1}})\ndisplaydh(HTML('cell 7.2'), metadata={'bridge': {'cell': 7.2}});",
 "# cell 16\ndh = displaydh(display_id=True, metadata={'bridge': {'cell': 16}})",
 "# cell 17\ndh = displaydh(HTML('cell 17'), metadata={'bridge': {'cell': 17}})",
 "# cell 19\ndisplaydh(JSON({'cell': 19}), metadata={'bridge': {'cell': 19}});",
 "# cell 24\nfrom itertools import count\nfrom bridget.helpers import display

In [ ]:
cb = FindCB('# find_me')
c = nb.process(cb).cells[0]
c

{'idx': 41,
 'cell_type': 'code',
 'source': '# cell 41\n# find_me',
 'id': 'X56sZmlsZQ==',
 'metadata': {'brd': {'id': '340489bf-419a-4da3-9b06-eb1b5cd6daba'}},
 'outputs': [],
 'execution_count': 7}

In [ ]:
WhereT = Literal['source', 'outputs', 'metadata', 'all']

@FC.patch
def find(self: NB, what:str|Callable[[NBCell], bool], where:WhereT='source') -> L:
    if isinstance(what, str): f = (lambda c: what in (str(getattr(c, where, '') if where != 'all' else c)))
    else: f = what
    return self.cells.filter(f)

In [ ]:
nb.find('# find_me')  # type: ignore

(#1) [{'idx': 41, 'cell_type': 'code', 'source': '# cell 41\n# find_me', 'id': 'X56sZmlsZQ==', 'metadata': {'brd': {'id': '340489bf-419a-4da3-9b06-eb1b5cd6daba'}}, 'outputs': [], 'execution_count': 7}]

In [ ]:
if in_vscode: val_at(c, 'metadata.brd')

In [ ]:
c = nb.cells[4]
c

{'idx': 4,
 'cell_type': 'code',
 'source': "# cell 4\nimport time\ntime.sleep(2)\ndisplaydh(HTML('cell 4'), metadata={'bridge': {'cell': 4}});",
 'id': 'W4sZmlsZQ==',
 'metadata': {'brd': {'id': '798f7f83-9b31-4e21-a36b-5520c6f72a2d'}},
 'outputs': [{'output_type': 'display_data',
   'data': {'text/html': 'cell 4',
    'text/plain': '<IPython.core.display.HTML object>'},
   'metadata': {'bridge': {'cell': 4}}}],
 'execution_count': 4}

In [ ]:
vals_at(c, 'outputs.*.data.text/plain')

('<IPython.core.display.HTML object>',)

In [ ]:
nb[2]

{'idx': 2,
 'cell_type': 'code',
 'source': '# cell 2\nprint(1)',
 'id': 'W2sZmlsZQ==',
 'metadata': {'brd': {'id': '92f3e304-fd32-4de2-badf-11577f9e7a4a'}},
 'outputs': [{'output_type': 'stream', 'name': 'stdout', 'text': '1\n'}],
 'execution_count': 2}

In [ ]:
vals_at(nb.cells, '*.outputs.*.data.text/plain')

(empty,
 empty,
 empty,
 ("'cell 3'",),
 ('<IPython.core.display.HTML object>',),
 ('<IPython.core.display.Javascript object>',),
 empty,
 ('<IPython.core.display.HTML object>', '<IPython.core.display.HTML object>'),
 ('<IPython.core.display.HTML object>',),
 ('<IPython.core.display.Javascript object>',),
 ('<IPython.core.display.Markdown object>',),
 ('<IPython.core.display.SVG object>',),
 ("IntSlider(value=12, description='cell')",),
 ('Output()',),
 ('<Figure size 400x267 with 1 Axes>',),
 ('<IPython.core.display.Image object>',),
 empty,
 ('<IPython.core.display.HTML object>',),
 empty,
 ('<IPython.core.display.JSON object>',),
 ("IntSlider(value=20, description='cell')",),
 empty,
 empty,
 empty,
 ('<IPython.core.display.HTML object>', ''),
 empty,
 ("'cell 26.1'", "'cell 26.2'"),
 ("'cell 27.1'", "'cell 27.2'"),
 empty,
 empty,
 empty,
 empty,
 empty,
 empty,
 empty,
 empty,
 empty,
 ('Output()',),
 empty,
 empty,
 empty,
 empty)

In [ ]:
vals_at(nb.cells, '*.source')

('# cell 0',
 '# cell 1\nimport time\nfrom itertools import count\n\nimport ipywidgets as W\nimport matplotlib.pyplot as plt\nfrom bridget.helpers import displaydh\nfrom IPython.display import HTML, Image, Javascript, JSON, DisplayHandle, clear_output\ncounter = count()',
 '# cell 2\nprint(1)',
 "# cell 3\ndisplaydh('cell 3', metadata={'bridge': {'cell': 3}});",
 "# cell 4\nimport time\ntime.sleep(2)\ndisplaydh(HTML('cell 4'), metadata={'bridge': {'cell': 4}});",
 '# cell 5\ndisplaydh(Javascript(\'console.log("cell 5")\'), metadata={\'bridge\': {\'cell\': 5}});',
 '# cell 6\nprint(6.1)\nprint(6.2)',
 "# cell 7\ndisplaydh(HTML('cell 7.1'), metadata={'bridge': {'cell': 7.1}})\ndisplaydh(HTML('cell 7.2'), metadata={'bridge': {'cell': 7.2}});",
 '%HTML\n<h3>cell 8</h3>',
 "%javascript\nconsole.log('cell 9')",
 '%markdown\ncell 10',
 '%SVG\n<svg width="100" height="20" xmlns="http://www.w3.org/2000/svg">\n  <text x="10" y="10">cell 11</text>\n</svg>',
 "# cell 12\nW.IntSlider(12, descriptio

In [ ]:
vals_at(nb.cells, '*.metadata.brd.id')[:10]  # type: ignore

('81c3e877-b59a-48f5-a1da-21894d928d4b',
 '717322f8-95fa-425c-839d-8b9e7d4ef921',
 '92f3e304-fd32-4de2-badf-11577f9e7a4a',
 'f2c19c18-a3f7-4acb-88e4-c7239178c401',
 '798f7f83-9b31-4e21-a36b-5520c6f72a2d',
 'a20081a9-4e7e-4301-be25-2a44f45bfc71',
 'e1ba8c0c-d248-44ff-9437-17482a711358',
 '7512abfe-f7d2-4047-a6b1-56aefbf9457e',
 '72d2a352-a5e3-4bb4-8e12-2302f9bac880',
 '39565bc0-3d14-4912-8220-e9dacb965ca4')

In [ ]:
vals_at(nb.cells, '*.outputs')[10:14]  # type: ignore

([{'output_type': 'display_data',
   'data': {'text/markdown': 'cell 10\n',
    'text/plain': '<IPython.core.display.Markdown object>'},
   'metadata': {}}],
 [{'output_type': 'display_data',
   'data': {'image/svg+xml': '<svg xmlns="http://www.w3.org/2000/svg" width="100" height="20">\n  <text x="10" y="10">cell 11</text>\n</svg>',
    'text/plain': '<IPython.core.display.SVG object>'},
   'metadata': {'__displayOpenPlotIcon': True}}],
 [{'output_type': 'execute_result',
   'data': {'application/vnd.jupyter.widget-view+json': {'version_major': 2,
     'version_minor': 0,
     'model_id': '6191b18d61a44fbd89eb132d22a28eab'},
    'text/plain': "IntSlider(value=12, description='cell')"},
   'execution_count': 12,
   'metadata': {}}],
 [{'output_type': 'display_data',
   'data': {'application/vnd.jupyter.widget-view+json': {'version_major': 2,
     'version_minor': 0,
     'model_id': 'a9408d195fb84649b38948893390ef10'},
    'text/plain': 'Output()'},
   'metadata': {}}])

## find


In [ ]:
#| exporti

def _there(where, what, op, c):
    res = vals_at(c, where)
    if res is empty: return False
    return any(op(r, what) for r in res)

In [ ]:
L(vals_at(nb, '*.outputs.*.data.application/json')).argwhere(lambda x: x is not empty)

(#1) [19]

In [ ]:
L(vals_at(nb, '*.outputs.*.name')).argwhere(lambda x: x is not empty)

(#2) [2,6]

In [ ]:
test_eq(_there('outputs.*.data', 'stdout', op.contains, nb.cells[7]), False)

In [ ]:
for idx in L(vals_at(nb, '*.outputs.*.name')).argwhere(lambda x: x is not empty):
    test_eq(_there('outputs.0.name', 'stdout', op.contains, nb[idx]), True)

In [ ]:
for idx in L(vals_at(nb, '*.metadata.brd.renderer')).argwhere(lambda x: x is not empty):
    test_eq(_there('metadata.brd.renderer', True, op.is_, nb[idx]), True)

In [ ]:
test_eq(any(_there('source', '# find_me', op.contains, c) for c in nb), True)

In [ ]:
#| export

@FC.patch
def find(self: NB, what:JSONVal|Callable[[NBCell], bool], where:str='source', op=op.contains) -> L:
    "Find cells matching `what` in `where` using `op`"
    # if isinstance(what, str): what = lambda c: False if (res := _F(c, where, None)) is None else what in res
    if not isinstance(what, Callable): what = partial(_there, where, what, op)
    return self.cells.filter(what)

In [ ]:
nbp = nb.process(slc=slice(31, None))
# nbp.find('# find_me').attrgot('source').map(shorten, mode='r')
vals_at(nbp.find('# find_me'), '*.source')

('# cell 41\n# find_me',)

In [ ]:
nb.find(True, 'metadata.brd.renderer', op.is_).attrgot('metadata')

(#2) [{'brd': {'id': 'e4bbb456-27fa-47d1-b95b-637b61e0e8f5', 'renderer': True}},{'brd': {'id': 'e7eb193d-f9fe-41b6-b817-1328963ff734', 'renderer': True}}]

In [ ]:
nb.find('stdout', 'outputs.0.name').attrgot('source')

(#2) ['# cell 2\nprint(1)','# cell 6\nprint(6.1)\nprint(6.2)']

In [ ]:
nb.find('stdout', 'outputs.*.name').attrgot('source')

(#2) ['# cell 2\nprint(1)','# cell 6\nprint(6.1)\nprint(6.2)']

## found

In [ ]:
@FC.patch
def found(self: NB, what:str|Callable[[NBCell], bool], where:WhereT='source', cbs:Callback|Sequence[Callback]=()) -> NB:
    return self.process(cbs or FindCB(what, where))

nb.found('# find_me').cells  # type: ignore

(#1) [{'idx': 41, 'cell_type': 'code', 'source': '# cell 41\n# find_me', 'id': 'X56sZmlsZQ==', 'metadata': {'brd': {'id': '340489bf-419a-4da3-9b06-eb1b5cd6daba'}}, 'outputs': [], 'execution_count': 7}]

In [ ]:
#| export

@FC.patch
def found(self: NB, 
        what:JSONVal|Callable[[NBCell], bool], where='source', op=op.contains, 
        cbs:Callback|Sequence[Callback]=()
    ) -> NB:
    return NB.from_NB(self, self.find(what, where, op)).process(cbs)

In [ ]:
nb.found('# find_me').cells.attrgot('source')

(#1) ['# cell 41\n# find_me']

In [ ]:
nb.found('#| hide').cells

(#1) [{'idx': 13, 'cell_type': 'code', 'source': "#| hide\n# cell 13\nw = W.Output()\ndisplay(w)\nwith w:\n    print('cell 13.1')\n    display('cell 13.2')\n    display(HTML('cell 13.3'))", 'id': 'X16sZmlsZQ==', 'metadata': {'brd': {'id': '8af15850-3d11-4498-a037-ce8b1ac2a161'}}, 'outputs': [{'output_type': 'display_data', 'data': {'application/vnd.jupyter.widget-view+json': {'version_major': 2, 'version_minor': 0, 'model_id': 'a9408d195fb84649b38948893390ef10'}, 'text/plain': 'Output()'}, 'metadata': {}}], 'execution_count': 13}]

In [ ]:
class StripDirectives(Callback):
    @property
    def nb(self): return self._nb
    def before_iter(self, istat): self._nb = NB.from_NB(istat.context, [])
    def on_iter(self, _, cell):
        self._nb.cells.append(c := cell.copy())
        extract_directives(c)

display([*shortens(nb.found('#| hide').source, 'r')])
display([*shortens(nb.found('#| hide', cbs=StripDirectives()).source, 'r')])

['#| hide\n# cell 13\nw = W.Output()\ndisplay…']

['# cell 13\nw = W.Output()\ndisplay(w)\nwith…']

In [ ]:
def strip_nbdev_dirs_(nb:NB):
    f = FC.compose(copycell, TZ.curried.do(extract_directives))
    return NB.from_NB(nb, nb.cells.map(f))

list(shortens(nb.found('#| hide').pipe((strip_nbdev_dirs_,)).source, 'r'))

['# cell 13\nw = W.Output()\ndisplay(w)\nwith…']

----
<!-- # Colophon -->

In [ ]:
#| hide
#| eval: false

import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean

In [ ]:
#| hide
#| eval: false

if FC.IN_NOTEBOOK:
    nb_path = '07_nb.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)